# Ensemble Learning: MNIST

# Question 1

## 1. Load Dataset

In [1]:
# Load MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')

In [2]:
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
mnist.data

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
mnist.target

0        5
1        0
2        4
3        1
4        9
        ..
69995    2
69996    3
69997    4
69998    5
69999    6
Name: class, Length: 70000, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

## 2. Data Preprocessing

In [5]:
# Split features and target
X = mnist.data
y = mnist.target

In [6]:
# Split training and test set
X_train, y_train = X[:50000], y[:50000]
X_valid, y_valid = X[50000:60000], y[50000:60000]
X_test, y_test = X[60000:], y[60000:]

## 3. Training Individual Classifier

In [7]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

rf_clf = RandomForestClassifier(random_state=42)
et_clf = ExtraTreesClassifier(random_state=42)
svm_clf = SVC(random_state=42)

estimators = [rf_clf, et_clf, svm_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the RandomForestClassifier(random_state=42)
Training the ExtraTreesClassifier(random_state=42)
Training the SVC(random_state=42)


In [8]:
[estimator.score(X_valid, y_valid) for estimator in estimators]

[0.9736, 0.9743, 0.9802]

## 4. Ensemble into Voting Classifier

### 4.1 Hard Voting

In [9]:
from sklearn.ensemble import VotingClassifier

named_estimators = [
    ("rf_clf", rf_clf),
    ("et_clf", et_clf),
    ("svm_clf", svm_clf),
]

voting_clf = VotingClassifier(named_estimators)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf_clf',
                              RandomForestClassifier(random_state=42)),
                             ('et_clf', ExtraTreesClassifier(random_state=42)),
                             ('svm_clf', SVC(random_state=42))])

In [10]:
voting_clf.score(X_valid, y_valid)

0.9778

### 4.2 Soft Voting

In [11]:
voting_clf = VotingClassifier(named_estimators, voting='soft')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf_clf',
                              RandomForestClassifier(random_state=42)),
                             ('et_clf', ExtraTreesClassifier(random_state=42)),
                             ('svm_clf', SVC(random_state=42))],
                 voting='soft')

# Question 2

In [16]:
from sklearn.ensemble import StackingClassifier

stack_clf = StackingClassifier(
    estimators=[
        ('rf_clf', RandomForestClassifier(random_state=42)),
        ('et_clf', ExtraTreesClassifier(random_state=42)),
        ('svm_clf', SVC(random_state=42))
    ],
    final_estimator=RandomForestClassifier(random_state=42),
    cv=5
)

stack_clf.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('rf_clf',
                                RandomForestClassifier(random_state=42)),
                               ('et_clf',
                                ExtraTreesClassifier(random_state=42)),
                               ('svm_clf', SVC(random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=42))